# Visualize and analyze data

## Package imports

In [ ]:
import sys

# installing needed packages - this may take a few minutes
#!conda install -c conda-forge --yes --prefix {sys.prefix} laspy
try:
    import shapely
    import shapely.geometry
    import folium
except ModuleNotFoundError:
    !conda install --yes --prefix {sys.prefix} shapely
    !conda install --yes --prefix {sys.prefix} folium
    import shapely
    import shapely.geometry
    import folium


try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    !conda install --yes --prefix {sys.prefix} matplotlib
    import matplotlib.pyplot as plt
#import laspy
import numpy as np

sys.path.append("..")
from pytreedb import db

## Import database

Define (local) MongoDB connection and import database from URL

In [ ]:
mydbfile = "my_pytree.db"
db_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/data.db"
mydb = db.PyTreeDB(dbfile=mydbfile)
mydb.import_db(db_url, overwrite=True)

## Show data on a map

First, let's query the database and select only trees for which terrestrial laser scanning (TLS) data is available. 

In [ ]:
res = mydb.query({"properties.data.mode": "TLS"})

To display the filtered trees on the map, we retrieve their locations, their IDs and their species.

In [ ]:
tree_locations = [tree["geometry"]["coordinates"] for tree in res]
tree_ids = [tree["properties"]["id"] for tree in res]
tree_species = [tree["properties"]["species"] for tree in res]

You can zoom in and click on the markers in the map created below to read out the ID and species of the respective tree.

In [ ]:
tooltip = "Click me!"
m = folium.Map([49.0077, 8.702], zoom_start=13)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

In [ ]:
tooltip = "Click me!"
m = folium.Map([49.0299, 8.4382], zoom_start=14)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

## Bar plots and Histograms

Available trees for each data source

In [ ]:
data = {
    "ALS": len(mydb.query({"properties.data.mode": "ALS"})),
    "ULS leaf-on": len(
        mydb.query({"$and": [{"properties.data.mode": "ULS"}, {"properties.data.canopy_condition": "leaf-on"}]})
    ),
    "ULS leaf-off": len(
        mydb.query({"$and": [{"properties.data.mode": "ULS"}, {"properties.data.canopy_condition": "leaf-off"}]})
    ),
    "TLS": len(mydb.query({"properties.data.mode": "TLS"})),
}

In [ ]:
names = list(data.keys())
values = list(data.values())

plots = plt.bar(range(len(data)), values, tick_label=names)
for bar in plots.patches:
    plt.annotate(
        format(bar.get_height(), "d"),
        (bar.get_x() + bar.get_width() / 2, bar.get_height()),
        ha="center",
        va="center",
        size=13,
        xytext=(0, -10),
        textcoords="offset points",
    )
plt.show()

Histograms of tree height/DBH/etc.

## Venn diagram?

## Scatter plot of metrics

Plot DBH against tree height

Compare crown diameter to tree height ratio for different species

Scatter plots of metrics obtained from different sources

## 3D plots of point clouds

In [ ]:
# pypotree / PPTK / Open3D / PyntCloud / matplotlib ?